מגישים: 
אורי ביטון-213868797
דניאל גולדשמיד-209328244

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import numpy as np

In [18]:
page=1
cars=[]




while True:
    try:
        idx_url = "https://www.ad.co.il/car?sp261=13907&rp264=1980,2016&pageindex=" + str(page)
        html = requests.get(idx_url)
        if html.status_code!=200:
            break
        content = html.content
        soup = BeautifulSoup(content, 'html.parser')
        only_cars=soup.find("div",class_="cards-wrap s m l")


        #links:
        tags=only_cars.findAll('a')
        links_ad=[]
        for tag in tags:
            links_ad.append(tag.get('href', None))
        #נתונים על כל רכב:

        for i in links_ad:
            car_one=[]
            url = "https://www.ad.co.il"+i 
            response = requests.get(url)
            results_page = BeautifulSoup(response.content, 'html.parser')
            name=results_page.find('h1','mb-3').text
            car_one.append(name)
            info=results_page.find_all('h2','card-title')        
            try:
                price = int(info[1].text.replace('₪', '').replace(',', ''))
                car_one.append(price)
            except:
                car_one.append(None)

            #table
            table=results_page.find('table','table table-sm mb-4')
            dct={'שנה': None,
                 'יד': None,
                 'ת. הילוכים': None,
                 'נפח': None,
                 'סוג מנוע': None,
                 'ק"מ': None,
                 'טסט עד': None,
                 'צבע': None,
                 'בעלות קודמת': None,
                 'בעלות נוכחית': None,
                 'אזור': None,
                 'עיר': None}
            lst=[]
            data={}
            counter=0
            for td in table.find_all('td'):
                    key = td.text.strip()
                    lst.append(td.text.strip())  
            for i in range(0,len(lst),2):
                dct[lst[i]]=lst[i+1]
            for key in dct:
                car_one.append(dct[key])
            #num of pics
            car_one.append(len(results_page.find_all('a',class_='post-1')))
            #taarih
            dates=results_page.find_all('div',class_='px-3')
            for date in dates:
                car_one.append(date.text.split(' ')[-1])
            #teur
            description=results_page.find('p',class_='text-word-break')
            if description:
                car_one.append(description.get_text().strip())

            else:
                car_one.append(None)

            cars.append(car_one)
        page+=1
    
    except:
        break
        
            


        
      
        


In [19]:
#we had a problem with couple of speciffic description
for i in cars:
    if i[17]=='רכב':
        del i[17]

In [20]:
df=pd.DataFrame(cars)

In [21]:
columns=['Model','Price','Year','Hand','Gear','Capacity_engine','Engine_type','Km','Test','Color','Prev_ownership','Curr_ownership','Area','City','Pic_num','Cre_date','Repub_date','Description']
df.columns=columns

שינוי TYPE:

Int:

In [6]:
df['Year']=pd.to_numeric(df['Year'], errors='coerce').astype('Int64')
df['Hand']=pd.to_numeric(df['Hand'], errors='coerce').astype('Int64')
for i in range(0,len(df['Km'])):
    if df['Km'][i]!=None:
        df['Km'][i]=int(df['Km'][i].replace(',',''))
df['Capacity_engine'] = pd.to_numeric(df['Capacity_engine'].str.replace(',',''), errors='coerce').astype('Int64')


df['Pic_num']=pd.to_numeric(df['Pic_num'], errors='coerce').astype('Int64')


<ipython-input-6-45b37da2dd4d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Km'][i]=int(df['Km'][i].replace(',',''))


Date time:

In [7]:
df['Cre_date'] = pd.to_datetime(df['Cre_date'])
df['Repub_date'] = pd.to_datetime(df['Repub_date'])

category:

In [8]:
#engine type
lst=['בנזין', 'דיזל','גז','היבריד', 'חשמלי']
df['Engine_type'].unique()
for i in range(len(df['Engine_type'])):
    if df['Engine_type'][i]=='טורבו דיזל':
        df['Engine_type'][i]='דיזל'
    if df['Engine_type'][i] not in lst:
        df['Engine_type'][i]=None
df['Engine_type'] = df['Engine_type'].astype('category')

<ipython-input-8-03733a2d39f1>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Engine_type'][i]='דיזל'


In [9]:
#Gear
unique_values, counts=np.unique(df['Gear'],return_counts=True)
value_counts = dict(zip(unique_values, counts))
lst1=list(value_counts.keys())
for i in range(len(df['Gear'])):
    if df['Gear'][i] not in lst1:
        df['Gear'][i]==None
df['Gear'] = df['Gear'].astype('category')       

In [10]:
#prev ownership:
lst2=list(df['Prev_ownership'].unique())
for i in range(len(df['Prev_ownership'])):
    if df['Prev_ownership'][i] not in lst2:
        df['Prev_ownership'][i]==None
df['Prev_ownership'] = df['Prev_ownership'].astype('category')


In [11]:
#curr ownership:
for i in range(len(df['Curr_ownership'])):
    if df['Curr_ownership'][i] not in lst2:
        df['Curr_ownership'][i]==None
df['Curr_ownership'] = df['Curr_ownership'].astype('category')

Test:

In [12]:

import pandas as pd
from calendar import monthrange

# דוגמה של DataFrame עם עמודת תאריך בפורמט "MM/YYYY"


# פונקציה להמיר תאריך בפורמט "MM/YYYY" ליום האחרון של אותו חודש
def get_last_day_of_month(date_str):
    if date_str is None:
        return None
    month, year = map(int, date_str.split('/'))
    last_day = monthrange(year, month)[1]
    return f"{year}-{month:02d}-{last_day}"

# הוספת עמודה חדשה עם היום האחרון של כל חודש
df['Test'] = df['Test'].apply(get_last_day_of_month)
df['Test']=pd.to_datetime(df['Test'])



In [15]:
import pandas as pd
from datetime import datetime
df['Test'] = df['Test'].dt.strftime('%Y/%m/%d')
df['Test']=pd.to_datetime(df['Test'])
today = pd.to_datetime(datetime.today().date())
df['Test']= ( df['Test']-today).dt.days
df['Test'] = pd.to_numeric(df['Test'], errors='coerce').astype('Int64')
df['Test']

0       170
1      <NA>
2      <NA>
3      <NA>
4       140
       ... 
144    <NA>
145    <NA>
146    <NA>
147    <NA>
148    <NA>
Name: Test, Length: 149, dtype: Int64

In [13]:
lil=[]
for i in range(len(df['Price'])):
    lil.append('סקודה')
df.insert(0,'Manufactor',lil)


In [17]:
df

,Manufactor,Model,Price,Year,Hand,Gear,Capacity_engine,Engine_type,Km,Test,Color,Prev_ownership,Curr_ownership,Area,City,Pic_num,Cre_date,Repub_date,Description
0,סקודה,סקודה ראפיד (2013),26000.0,2013,3,אוטומטית,1400,בנזין,169000,170,כחול,פרטית,פרטית,None,עראבה,3,2024-04-29,2024-04-29,Skoda \nדגם : Rapid \nשנת יצור : 2013 \nק״מ : ...
1,סקודה,סקודה אוקטביה (2013),21000.0,2013,3,אוטומטית,1800,בנזין,185000,<NA>,כחול כהה,פרטית,פרטית,באר שבע והסביבה,באר שבע,5,2023-04-10,2024-04-25,"רכב סקודה אוקטביה 1800 טורבו , יד 3 , ק""מ 185 ..."
2,סקודה,סקודה רומסטר (2015),24000.0,2015,2,אוטומטית,1200,בנזין,240000,<NA>,לבן,פרטית,פרטית,באר שבע והסביבה,באר שבע,5,2023-06-01,2024-04-21,"סקודה רומסטר 1200 טורבו , יד 2 , ק""מ 240 אלף\n..."
3,סקודה,סקודה פאביה (2012),11500.0,2012,4,ידנית,1400,בנזין,225000,<NA>,אפור,פרטית,פרטית,ירושלים והסביבה,ירושלים,5,2024-12-04,2024-04-17,רכב חסכוני.\nמוביליי.\nצמיגים חדשים.\nמצבר חדש...
4,סקודה,סקודה רומסטר (2012),25000.0,2012,4,אוטומטית,1200,בנזין,200000,140,כסוף מטאלי,פרטית,פרטית,None,סח'נין,9,2024-03-31,2024-03-31,סקודה ייטי 2012 גיר אוטומט 1200 טורבו\nיד 04 פ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,סקודה,סקודה פאביה (2016),52000.0,2016,2,אוטומטית,1200,בנזין,149000,<NA>,כחול בהיר מטאלי,ליסינג,פרטית,חיפה וחוף הכרמל,טירת כרמל,8,2022-08-01,2022-01-27,רכב שמור אמין וחסכוני מאוד ללא תאונות דגם סטיי...
145,סקודה,סקודה רומסטר\r\n (2013),20000.0,2013,1,ידנית,1300,בנזין,174000,<NA>,None,NaN,NaN,None,אבני איתן,0,2022-05-07,2022-05-07,יד ראשונה במצב מעולה \r\nקילומטראז׳ נמוך\r\nחס...
146,סקודה,סקודה ראפיד (2015),38000.0,2015,3,אוטומטית,1395,בנזין,162000,<NA>,None,NaN,NaN,רמלה - לוד,לוד,0,2022-12-05,2022-12-05,כמו חדש
147,סקודה,סקודה אוקטביה (2012),29000.0,2012,3,אוטומטית,1200,בנזין,212,<NA>,כחול,פרטית,פרטית,ירושלים והסביבה,ירושלים,0,2021-11-20,2021-11-20,None


In [25]:
file_path = 'data_part1.csv'

# Save the DataFrame to a CSV file
df.to_csv(file_path, index=False,encoding='utf-8-sig')